In [1]:
import warnings
import pandas as pd
import numpy as np
from scipy.linalg import inv, pinv, LinAlgError

# Framework

The system is considered in a steady-state

![static mfa](../img/static_mfa.jpg)   
Source: Lupton & Allwood (2018)
  
  
$\begin{cases}q_j + \sum\limits_{i=0}^nz_{ij} = \sum\limits_{i=0}^mz_{jk} \\ z_{jk} = \text{TC}_{jk} \begin{pmatrix}q_j + \sum\limits_{i=0}^nz_{ij}\end{pmatrix}\end{cases}$

# Utility functions

In [2]:
def get_data(filepath, xlsx_map):
    xlsx_file = pd.read_excel(filepath, sheet_name=None)
    TC = xlsx_map["transfer_coefficient"]
    FLOWS = xlsx_map["flows"]
    df_TC = xlsx_file[TC["sheet"]].rename(columns=xlsx_map["transfer_coefficient"])
    df_flows = xlsx_file[FLOWS["sheet"]].rename(columns=xlsx_map["flows"])
    return [df_TC, df_flows]


def solve(_in, _out, y, coeff, singular=False):
    """X = AX + y"""
    A = coeff[:, None] * (_in == _out[:, None])
    I = np.eye(len(A))
    L = pinv(I - A) if singular else inv(I - A)
    return L @ y


def solve_subsystem(df, *columns):
    _from, _to, _val, _pct = columns
    arr_from = df[_from].values
    arr_to = df[_to].values
    arr_pct = df[_pct].fillna(0).values
    arr_val = df[_val].fillna(0).values
    try:
        result = solve(_in=arr_to, _out=arr_from, y=arr_val, coeff=arr_pct)
    except LinAlgError as err:
        if 'singular matrix' in str(err):
            idx = list(df.set_index(list(df.columns.difference(columns))).index.unique())
            warnings.warn(f"singular matrix detected at {idx}. "
                        "Defaulted on scipy.linalg.pinv for calculations")
            result = solve(_in=arr_to, _out=arr_from, y=arr_val, coeff=arr_pct, singular=True)
        else:
            raise
    return list(zip(arr_from, arr_to, result))


def explode(df, *columns):
    _from, _to, _val, _pct = columns
    df = df.explode()
    return pd.DataFrame(
        data=df.values.tolist(), 
        index=df.index, 
        columns=[_from, _to, _val],
    )


def compute_system(filepath, xlsx_map, *columns):
    df_TC, df_flows = get_data(filepath, xlsx_map)
    other_columns = df_flows.columns.union(df_TC.columns).difference(columns)
    system_state = (
        pd.concat([df_flows, df_TC])
        .groupby(list(other_columns))
        .apply(solve_subsystem, *columns)
    )
    return explode(system_state, *columns)

# Application

In [3]:
FILEPATH = "../data/dummy_data.xlsx"

In [4]:
FROM, TO, VAL, PCT = ('_from', '_to', '_val', '_pct')

XLSX_MAP = {
    # flows
    "flows": {"sheet": "inputs", 
        "processFrom": FROM, "processTo": TO, "value": VAL},
    # TCs
    "transfer_coefficient": { "sheet": "trans_coeff", 
        "processFrom": FROM, "processTo": TO, "tc": PCT},
}

In [5]:
result = compute_system(FILEPATH, XLSX_MAP, FROM, TO, VAL, PCT)
result

_from           _to  _val
country material  year                                  
DE      Aluminium 2020           EXT      Leaching  10.0
                  2020  Inciniration   Fabrication   0.0
                  2020  Inciniration       Melting   0.0
                  2020  Inciniration       Sorting   0.0
                  2020  Inciniration     Recycling   0.0
...                              ...           ...   ...
UK      Zinc      2023   Landfilling     Recycling   0.0
                  2023   Landfilling      Leaching   0.0
                  2023      Leaching       Sorting   0.0
                  2023      Leaching  Inciniration   0.0
                  2023      Leaching   Fabrication   0.0

[4398 rows x 3 columns]

# References

Lupton, R. C., & Allwood, J. M. (2018). Incremental material flow analysis with Bayesian inference. Journal of Industrial Ecology, 22(6), 1352-1364.